In [1]:
import numpy as np
import pandas as pd
import os
import yfinance as yf
import datetime
import requests
from bs4 import BeautifulSoup as soup

current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month
years_forward = 4


In [2]:
def get_name(i):
	name = i["shortName"]
	return name


def get_current_price(i):
	current_price = i["currentPrice"]
	return current_price


def get_chart(h):
	chart_dates = h.index
	chart_values = h["Close"].values

	return chart_dates, chart_values


def get_details(tick):
	i = yf.Ticker(tick).info

	b = yf.Ticker(tick).balance_sheet
	b = b[b.columns[::-1]]
	b.columns = pd.DatetimeIndex(b.columns).year

	c = yf.Ticker(tick).cashflow
	c = c[c.columns[::-1]]
	c.columns = pd.DatetimeIndex(c.columns).year

	f = yf.Ticker(tick).financials
	f = f[f.columns[::-1]]
	f.columns = pd.DatetimeIndex(f.columns).year

	a = yf.Ticker(tick).analysis

	h = yf.Ticker(tick).history(period="5y")

	return i, b, c, f, a, h

In [4]:
tick = "AAPL"
i, b, c, f, a, h = get_details(tick)
name = get_name(i)
current_price = get_current_price(i)

In [5]:
def get_free_cashflow(c):
	CPO = c.loc["Total Cash From Operating Activities"].to_frame().T
	CAPEX = c.loc["Capital Expenditures"].to_frame().T
	FCF = (CPO.iloc[0] + CAPEX.iloc[0]).to_frame().T
	FCF.index = ["Free Cashflow"]
	FCF = CPO.append([CAPEX, FCF])
	return FCF

In [6]:
FCF = get_free_cashflow(c)
FCF

,2018,2019,2020,2021
Total Cash From Operating Activities,7.743400e+10,6.939100e+10,8.067400e+10,1.040380e+11
Capital Expenditures,-1.331300e+10,-1.049500e+10,-7.309000e+09,-1.108500e+10
Free Cashflow,6.412100e+10,5.889600e+10,7.336500e+10,9.295300e+10


In [7]:

def get_net_income(c, FCF):
	net_income = c.loc["Net Income"].to_frame().T
	net_income.loc["Free Cash Flow to Equity"] = FCF.loc["Free Cashflow"]
	net_income.loc["FCFE/Net Income"] = (FCF.loc["Free Cashflow"] / net_income.loc["Net Income"])
	net_income.loc["FCFE/Net Income"] = net_income.loc["FCFE/Net Income"].round(2) * 100
	
	median_FFCE = net_income.loc["FCFE/Net Income"].median()
	
	return median_FFCE, net_income

In [9]:
median_FFCE, net_income = get_net_income(c, FCF)
median_FFCE

107.5

In [10]:
net_income

,2018,2019,2020,2021
Net Income,5.953100e+10,5.525600e+10,5.741100e+10,9.468000e+10
Free Cash Flow to Equity,6.412100e+10,5.889600e+10,7.336500e+10,9.295300e+10
FCFE/Net Income,1.080000e+02,1.070000e+02,1.280000e+02,9.800000e+01


In [11]:
def get_total_revenue(f, a):
	total_revenue = f.loc["Total Revenue"].to_frame().T
	total_revenue[current_year] = a["Revenue Estimate Low"].loc["0Y"]
	total_revenue[current_year+1] = a["Revenue Estimate Low"].loc["+1Y"]

	growths = [np.nan]
	for year1, year2 in zip(total_revenue, total_revenue.iloc[:, 1:]):
		y1 = total_revenue[year1].iloc[0]
		y2 = total_revenue[year2].iloc[0]
		growth = round((y2 - y1)/y1*100, 2)
		growths.append(growth)

	total_revenue.loc["Revenue Growth Rate"] = growths
	median_growth = total_revenue.loc["Revenue Growth Rate"].median()

	return median_growth, total_revenue

In [12]:
median_growth, total_revenue = get_total_revenue(f, a)
median_growth

5.51

In [13]:
total_revenue

,2018,2019,2020,2021,2022,2023
Total Revenue,2.65595e+11,2.60174e+11,2.74515e+11,3.65817e+11,3.872000e+11,3.937660e+11
Revenue Growth Rate,NaN,-2.04,5.51,33.26,5.850000e+00,1.700000e+00


In [14]:
def get_net_income_margins(total_revenue, net_income):
	net_income_margins = pd.concat([total_revenue.loc["Total Revenue"].to_frame().T, net_income.loc["Net Income"].to_frame().T])
	
	net_income = []
	for year in net_income_margins.columns:
		revenue = net_income_margins[year].loc["Total Revenue"]
		netincome = net_income_margins[year].loc["Net Income"]
		net = round(netincome/revenue * 100, 2)
		net_income.append(net)
	net_income_margins.loc["Net Income Margins"] = net_income
	
	free_cashflow_rate_median = net_income_margins.loc["Net Income Margins"].median()

	return free_cashflow_rate_median, net_income_margins

In [15]:
free_cashflow_rate_median, net_income_margins = get_net_income_margins(total_revenue, net_income)
free_cashflow_rate_median

21.825

In [16]:
net_income_margins

,2018,2019,2020,2021,2022,2023
Total Revenue,2.65595e+11,2.60174e+11,2.74515e+11,3.65817e+11,3.872e+11,3.93766e+11
Net Income,5.9531e+10,5.5256e+10,5.7411e+10,9.468e+10,NaN,NaN
Net Income Margins,22.41,21.24,20.91,25.88,NaN,NaN


In [17]:
def get_income_statement_ahead(net_income_margins, median_growth, median_FFCE, FCF, free_cashflow_rate_median):
	income_statement_numbers = net_income_margins.copy()
	income_statement_numbers = net_income_margins.drop(index="Net Income Margins", axis=0)
	income_statement_numbers = income_statement_numbers.append(FCF.loc["Free Cashflow"])

	for y in range(current_year, current_year+years_forward):
		rev = income_statement_numbers[y-1].loc["Total Revenue"] * (1+ (median_growth/ 100))
		n = rev * free_cashflow_rate_median / 100
		fcf = n * median_FFCE / 100
		income_statement_numbers[y] = [rev, n, fcf]
		
	return income_statement_numbers

In [18]:
income_statement_numbers = get_income_statement_ahead(net_income_margins, median_growth, median_FFCE, FCF, free_cashflow_rate_median)
income_statement_numbers

,2018,2019,2020,2021,2022,2023,2024,2025
Total Revenue,2.65595e+11,2.60174e+11,2.74515e+11,3.65817e+11,3.859735e+11,4.072407e+11,4.296796e+11,4.533550e+11
Net Income,5.9531e+10,5.5256e+10,5.7411e+10,9.468e+10,8.423872e+10,8.888027e+10,9.377758e+10,9.894472e+10
Free Cashflow,6.4121e+10,5.8896e+10,7.3365e+10,9.2953e+10,9.055662e+10,9.554629e+10,1.008109e+11,1.063656e+11


In [20]:
def scrap_Rf():
	if (os.path.isfile('./TNX.txt')):
		with open("./TNX.txt", "r") as f:
			file_date = f.readline().strip()
			Rf = float(f.readline().strip())
			if file_date == f"{current_month}/{current_year}":
				return Rf

	page = requests.get("https://finance.yahoo.com/bonds")
	content = soup(page.content, "html.parser")
	Rf = float(content.find("fin-streamer", {"data-symbol":"^TNX"}).text)
	if Rf:
		with open("./TNX.txt", "w") as f:
			f.write(f"{current_month}/{current_year}\n{Rf}")
	else: 
		Rf = 3
	return Rf

In [25]:
def get_WACC(f, b, i):
	t = f[current_year-1].loc["Income Tax Expense"]/f[current_year-1].loc["Income Before Tax"]
	t = round(t, 4)

	rd = (round(abs(f[current_year-1].loc["Interest Expense"])*100 / 
			 (b[current_year-1].loc["Short Long Term Debt"]+b[current_year-1].loc["Long Term Debt"]),2))

	Rf = scrap_Rf()

	B = i["beta"] if i["beta"] else 0.8

	Rm = 10

	print(t, rd, Rf, B)

	Ra = Rf + B*(Rm - Rf)
	re = Ra

	total = i["totalDebt"] + i["marketCap"]
	wd = i["totalDebt"]*100/total
	we = i["marketCap"]*100/total

	WACC = round((wd*rd*(1-t) + we*re)/100,2)
	required_return = WACC
	r = required_return/100

	return r

In [26]:
r = get_WACC(f, b, i)
r

0.133 2.23 3.239 1.20009


0.1085

In [27]:

def get_terminal_value(i, income_statement_numbers, r):
	perpetual_growth = 2.5
	g = perpetual_growth / 100

	FCFE0 = income_statement_numbers[current_year+years_forward-1].loc["Free Cashflow"]
	terminal_value = (FCFE0*(1+g))/(r-g)
	terminal_value = round(terminal_value)

	return terminal_value, g


In [28]:
terminal_value, g = get_terminal_value(i, income_statement_numbers, r)
terminal_value

1305685203424

In [29]:

def get_discount_factors(income_statement_numbers, r):
	cols_to_drop = list(filter(lambda x: x < current_year, income_statement_numbers.columns))
	discount_factors = income_statement_numbers.loc["Free Cashflow"].to_frame().T.drop(cols_to_drop, axis=1)

	discount_facs = []
	discount_val = []
	for (j, year) in enumerate(discount_factors.columns):
		discount = (1 + r)**(j+1)
		cashflow = discount_factors[year].loc["Free Cashflow"]
		discount_facs.append(discount)
		discount_val.append(cashflow/discount)
		
	discount_factors.loc["Discount Factor"] = discount_facs
	discount_factors.loc["PV of Future Cash Flow"] = discount_val
	return discount_factors

In [30]:
discount_factors = get_discount_factors(income_statement_numbers, r)
discount_factors

,2022,2023,2024,2025
Free Cashflow,9.05566e+10,9.55463e+10,1.00811e+11,1.06366e+11
Discount Factor,1.1085,1.22877,1.36209,1.50988
PV of Future Cash Flow,8.16929e+10,7.77575e+10,7.40117e+10,7.04463e+10


In [45]:
def get_intrinsic_value(i, terminal_value, discount_facs, margin_of_safety = 0.2):
	shares_outstanding = i["sharesOutstanding"]
	PV_of_terminal_value = round(terminal_value * discount_factors[current_year+years_forward-1].loc["Discount Factor"])
	todays_value = round(discount_factors.loc["PV of Future Cash Flow"].sum() + PV_of_terminal_value)
	
	fair_value_of_equity = todays_value/shares_outstanding
	intrinsic_value = fair_value_of_equity*(1-margin_of_safety)

	return intrinsic_value

In [46]:
intrinsic_value = get_intrinsic_value(i, terminal_value, discount_factors, margin_of_safety = 0.2)
intrinsic_value

112.46512312098753